In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

adult_census = pd.read_csv('../datasets/adult-census.csv')

target_name = 'class'
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, 'education-num'])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(
    handle_unknown='use_encoded_value', 
    unknown_value=-1
)

preprocessor = ColumnTransformer(
    [
        ('categorical', categorical_preprocessor, selector(dtype_include=object)),
    ],
    remainder='passthrough'
)

Xây dựng mô hình sử dụng `HistGradientBoostingClassifier`:

In [5]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('classifier', HistGradientBoostingClassifier(random_state=42)),
    ]
)

In [8]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000026B7F5C6BA0>)])),
                ('classifier',
                 HistGradientBoostingClassifier(random_state=42))])

In [9]:
for parameter in model['classifier'].get_params():
    print(parameter)

categorical_features
class_weight
early_stopping
interaction_cst
l2_regularization
learning_rate
loss
max_bins
max_depth
max_features
max_iter
max_leaf_nodes
min_samples_leaf
monotonic_cst
n_iter_no_change
random_state
scoring
tol
validation_fraction
verbose
warm_start


Sử dụng mô hình đã định nghĩa trước đó (gọi là `model`) và sử dụng hai vòng lặp lồng nhau, thực hiện tìm kiếm sự kết hợp các tham số tốt nhất cho các tham số `learning_rate` và `max_leaf_nodes`. Trong quá trình này, bạn cần huấn luyện và kiểm tra mô hình bằng cách thiết lập các tham số. Đánh giá mô hình nên được thực hiện bằng `cross_val_score` trên tập huấn luyện. Sử dụng các giá trị tìm kiếm tham số sau:

- `learning_rate` cho các giá trị 0.01, 0.1, 1 và 10. Tham số này điều khiển khả năng của mỗi cây mới trong việc sửa lỗi của dãy cây trước đó.

- `max_leaf_nodes` cho các giá trị 3, 10, 30. Tham số này điều khiển độ sâu của mỗi cây.

In [13]:
from sklearn.model_selection import cross_validate

for learning_rate in [0.01, 0.1, 1, 10]:
    for max_leaf_nodes in [3, 10, 30]:
        model['classifier'].set_params(
            learning_rate=learning_rate,
            max_leaf_nodes=max_leaf_nodes
        )

        cv_results = cross_validate(
            model,
            data_train,
            target_train,
        )

        scores = cv_results['test_score']

        print(
            f'Accuracy score via cross-validation with learning_rate={learning_rate} and max_left_nodes={max_leaf_nodes}:'
            f' {scores.mean():.3f} +/- {scores.std():.3f}'
        )



Accuracy score via cross-validation with learning_rate=0.01 and max_left_nodes=3: 0.797 +/- 0.002
Accuracy score via cross-validation with learning_rate=0.01 and max_left_nodes=10: 0.819 +/- 0.003
Accuracy score via cross-validation with learning_rate=0.01 and max_left_nodes=30: 0.848 +/- 0.004
Accuracy score via cross-validation with learning_rate=0.1 and max_left_nodes=3: 0.855 +/- 0.003
Accuracy score via cross-validation with learning_rate=0.1 and max_left_nodes=10: 0.869 +/- 0.003
Accuracy score via cross-validation with learning_rate=0.1 and max_left_nodes=30: 0.871 +/- 0.003
Accuracy score via cross-validation with learning_rate=1 and max_left_nodes=3: 0.862 +/- 0.007
Accuracy score via cross-validation with learning_rate=1 and max_left_nodes=10: 0.862 +/- 0.009
Accuracy score via cross-validation with learning_rate=1 and max_left_nodes=30: 0.859 +/- 0.003
Accuracy score via cross-validation with learning_rate=10 and max_left_nodes=3: 0.282 +/- 0.005
Accuracy score via cross-val

Sau đó, sử dụng tập kiểm tra để đánh giá mô hình bằng cách sử dụng các tham số tốt nhất mà chúng ta đã tìm thấy qua kiểm tra chéo (cross-validation). Bạn sẽ phải huấn luyện lại mô hình trên toàn bộ tập huấn luyện.

In [15]:
model.set_params(
    classifier__learning_rate=0.1,
    classifier__max_leaf_nodes=30,
)

model.fit(data_train, target_train)
predictions = model.predict(data_test)
print(f'Accuracy on test set: {model.score(data_test, target_test):.3f}')

Accuracy on test set: 0.879
